In [ ]:
!pip install geopy
!pip install folium

In [ ]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_csv="/content/drive/MyDrive/Colab Notebook/SNCF/tgvmax.db"

In [ ]:
import sqlite3
import csv
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import os

# Connexion à la base (elle sera créée si elle n'existe pas)
# We use a path that is likely to be writable in Colab
db_path = path_csv
conn = sqlite3.connect(r"C:\Users\shenc\OneDrive\Bureau\Polytech\MAM4\projetopendata\tgvmax.db")
cur = conn.cursor()

def SQL(a):
    cur.execute(a)
    return(cur.fetchall())

Now you can run the following cells to get user input and perform the distance calculation.

In [ ]:
#ville de départ destination disponible, horaire, temps de trajets
ville_dep=input("ville de départ : ")

pattern_1 = f"%{ville_dep}%"

sql = """
SELECT DISTINCT Origine, Destination,
       TIME(Heure_depart) AS heure_depart,
       TIME(Heure_arrivee) AS heure_arrivee,
       -- Calculate duration in seconds and convert to HH:MM:SS format
       strftime('%H:%M:%S', (julianday(Heure_arrivee) - julianday(Heure_depart)) * 86400, 'unixepoch') AS duree
FROM tgvmax_trajets
WHERE LOWER(Origine) LIKE LOWER(?)
"""
cur.execute(sql, (pattern_1,))
resultats = cur.fetchall()
print(resultats)

ville de départ : aubagne


OperationalError: no such table: tgvmax_trajets

In [ ]:
#ville de départ et ville d'arriver
ville_dest=input("destination : ")
print(ville_dep,ville_dest)

pattern_2 = f"%{ville_dest}%"

sql = """
SELECT DISTINCT Origine, Destination,
       TIME(Heure_depart) AS heure_depart,
       TIME(Heure_arrivee) AS heure_arrivee,
       -- Calculate duration in seconds and convert to HH:MM:SS format
       strftime('%H:%M:%S', (julianday(Heure_arrivee) - julianday(Heure_depart)) * 86400, 'unixepoch') AS duree
FROM tgvmax_trajets
WHERE LOWER(Origine) LIKE LOWER(?) AND LOWER(Destination) LIKE LOWER(?)
"""
cur.execute(sql, (pattern_1,pattern_2))
resultats2 = cur.fetchall()
print(resultats2)

KeyboardInterrupt: Interrupted by user

In [ ]:
def distance_villes(ville_dep,ville_dest):
  #détermination de la distance entre ville_dep et ville_dest
  # Création d'un géocodeur (utilise OpenStreetMap)
  geolocator = Nominatim(user_agent="distance_calculator")

  # Recherche des coordonnées GPS
  loc_dep = geolocator.geocode(ville_dep)
  loc_dest = geolocator.geocode(ville_dest)

  distance_km = None # Initialize distance_km

  if loc_dep and loc_dest:
      coord_dep = (loc_dep.latitude, loc_dep.longitude)
      coord_dest = (loc_dest.latitude, loc_dest.longitude)

      # Calcul de la distance à vol d'oiseau
      distance_km = geodesic(coord_dep, coord_dest).kilometers

  return(distance_km)

print(f"Distance entre {ville_dep} et {ville_dest} : {distance_villes(ville_dep,ville_dest):.2f} km")

NameError: name 'ville_dep' is not defined

In [ ]:
print(distance_villes("NICE VILLE","ROUEN"))

793.1889836747138


In [ ]:
distance_max=input("disantce maximal parcourue (km)")

#ville de départ destination disponible, horaire, temps de trajets
ville_dep=input("ville de départ : ")

pattern_1 = f"%{ville_dep}%"

sql = """
SELECT DISTINCT Origine, Destination
FROM tgvmax_trajets
WHERE LOWER(Origine) LIKE LOWER(?)
"""

cur.execute(sql, (pattern_1,))
resultats3 = cur.fetchall()
#print(resultats3)



disantce maximal parcourue (km)300
ville de départ : paris


In [ ]:
unique_cities = set()
for i in resultats3:
  unique_cities.add(i[0])
  unique_cities.add(i[1])

In [ ]:
import time
city_coordinates = {}
geolocator = Nominatim(user_agent="distance_calculator", timeout=10) # Increase timeout

for i, city in enumerate(unique_cities):
  try:
    # Add a small delay between requests to avoid overloading the server
    if i > 0:
        time.sleep(1) # Pause for 1 second between requests

    # Check for "(intramuros)" and replace with just the city name
    cleaned_city = city.replace(" (intramuros)", "")

    loc_dep = geolocator.geocode(cleaned_city)
    if loc_dep:
      city_coordinates[city] = (loc_dep.latitude, loc_dep.longitude)
    else:
      print(f"Could not geocode city: {city}") # Print cities that could not be geocoded

  except Exception as e:
    print(f"Error geocoding city {city}: {e}") # Print any other errors during geocoding
    continue # Continue with the next city if an error occurs

Could not geocode city: L'HOSPITALET PRES L'AND.
Could not geocode city: LAROQUEBROU BATIMENT VOYAGEURS
Could not geocode city: OUX CESANA CLAV SESTRIERE


In [ ]:
distances = []
for i in resultats3:
  origine = i[0]
  destination = i[1]
  origin_coords = city_coordinates.get(origine)
  destination_coords = city_coordinates.get(destination)

  distance_km = None
  if origin_coords and destination_coords:
    distance_km = geodesic(origin_coords, destination_coords).kilometers

  distances.append((origine, destination, distance_km))

In [ ]:
print(distances)

[('LE MANS', 'PARIS (intramuros)', 184.93429957019757), ('LE MANS', 'RENNES', 140.36962868319074), ('LE MANS', 'ST BRIEUC', 226.6595819118747), ('LE MANS', 'NANTES', 158.16707794651487), ('LE MANS', 'SABLE SUR SARTHE', 43.89539899237696), ('LE MANS', 'ANGERS SAINT LAUD', 82.67398997605073), ('LE MANS', 'MARNE LA VALLEE CHESSY', 214.03478030227106), ('LE MANS', 'LYON (intramuros)', 432.73570560083306), ('LE MANS', 'STRASBOURG', 563.909356499831), ('LE MANS', 'MASSY TGV', 172.57050258538712), ('LE MANS', 'CHAMPAGNE ARDENNE TGV', 310.4829718122808), ('LE MANS', 'VERSAILLES CHANTIERS', 168.20122797713253), ('LE MANS', 'CHARTRES', 107.52184956294452), ('LE MANS', 'PORNICHET', 207.64634087835452), ('LE MANS', 'VITRE', 105.97171759042467), ('LE MANS', 'MORLAIX', 305.743010702069), ('LE MANS', 'LORRAINE TGV', 453.68746370987606), ('LE MANS', 'LILLE (intramuros)', 359.01575617355365), ('LE MANS', 'TGV HAUTE PICARDIE', 282.2550713631029), ('LE MANS', 'LAVAL', 72.6868351945917), ('LE MANS', 'AIX 

In [ ]:
filtered_results = []
try:
    distance_max_float = float(distance_max)
    for origin, destination, distance_km in distances:
      if distance_km is not None and distance_km < distance_max_float:
        filtered_results.append((origin, destination))

    for result in filtered_results:
        print(result)

except ValueError:
    print("Invalid input for maximum distance. Please enter a number.")

('PARIS (intramuros)', 'REIMS')
('PARIS (intramuros)', 'CHARLEVILLE MEZIERES')
('PARIS (intramuros)', 'NANCY')
('PARIS (intramuros)', 'ISSOUDUN')
('PARIS (intramuros)', 'CHAMPAGNE ARDENNE TGV')
('PARIS (intramuros)', 'LENS')
('PARIS (intramuros)', 'LILLE (intramuros)')
('PARIS (intramuros)', 'ANGERS SAINT LAUD')
('PARIS (intramuros)', 'POITIERS')
('PARIS (intramuros)', 'METZ VILLE')
('PARIS (intramuros)', 'THIONVILLE')
('PARIS (intramuros)', 'CHATEAUROUX')
('PARIS (intramuros)', 'VIERZON')
('PARIS (intramuros)', 'LE MANS')
('PARIS (intramuros)', 'ST PIERRE DES CORPS')
('PARIS (intramuros)', 'LES AUBRAIS ORLEANS')
('PARIS (intramuros)', 'BLOIS   CHAMBORD')
('PARIS (intramuros)', 'ARGENTON SUR CREUSE')
('PARIS (intramuros)', 'BETHUNE')
('PARIS (intramuros)', 'LE CREUSOT MONTCEAU MONTCHANIN')
('PARIS (intramuros)', 'FUTUROSCOPE')
('PARIS (intramuros)', 'LUXEMBOURG')
('PARIS (intramuros)', 'NEVERS')
('PARIS (intramuros)', 'BOULOGNE VILLE')
('PARIS (intramuros)', 'SABLE SUR SARTHE')
('PARIS

In [ ]:
print(filtered_results)

[('LE MANS', 'PARIS (intramuros)'), ('LE MANS', 'RENNES'), ('LE MANS', 'NANTES'), ('LE MANS', 'SABLE SUR SARTHE'), ('LE MANS', 'ANGERS SAINT LAUD'), ('LE MANS', 'MASSY TGV'), ('LE MANS', 'VERSAILLES CHANTIERS'), ('LE MANS', 'CHARTRES'), ('LE MANS', 'VITRE'), ('LE MANS', 'LAVAL'), ('LE MANS', 'LA ROCHE SUR YON'), ('LE MANS', 'MASSY PALAISEAU'), ('LE MANS', 'ST NAZAIRE'), ('LE MANS', 'JUVISY'), ('LE MANS', "PONT DE RUNGIS AEROPORT D'ORLY"), ('LE MANS', 'DOL DE BRETAGNE'), ('LE MANS', 'ANCENIS')]


In [ ]:
import folium
from geopy.geocoders import Nominatim

# Get city name from user input
city_name_input = ville_dep.upper().strip()

# Geocode the city name to get coordinates
geolocator = Nominatim(user_agent="city_mapper")
location = geolocator.geocode(city_name_input)

if location:
    # Store the origin city coordinates
    origin_coords = (location.latitude, location.longitude)
    origin_city_key = city_name_input # Use the input as the key for simplicity

    # Create a map centered at the origin city
    m = folium.Map(location=origin_coords, zoom_start=10)

    # Add a red marker for the origin city
    folium.Marker(
        location=origin_coords,
        popup=origin_city_key,
        icon=folium.Icon(color='red')
    ).add_to(m)

    # Add a circle representing the maximum distance
    # Assuming distance_max_float and filtered_results are available from previous cells
    try:
        folium.Circle(
            location=origin_coords,
            radius=distance_max_float * 1000,  # Radius in meters
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.1
        ).add_to(m)

        # Add markers for the filtered destination cities
        for origin, destination in filtered_results:
            destination_coords = city_coordinates.get(destination)
            if destination_coords:
                folium.Marker(
                    location=destination_coords,
                    popup=destination,
                    icon=folium.Icon(color='green')
                ).add_to(m)

    except NameError:
        print("Please make sure 'distance_max_float', 'filtered_results', and 'city_coordinates' are defined by running previous cells.")


    # Display the map
    display(m)
else:
    print(f"Could not find coordinates for the origin city: {city_name_input}")

In [ ]:
!pip install geodatamine

ERROR: Could not find a version that satisfies the requirement geodatamine (from versions: none)
ERROR: No matching distribution found for geodatamine


In [ ]:
from geopy.geocoders import Nominatim
import folium

city_name = input("Enter the name of the city: ")

Enter the name of the city: antibes


Next, we'll use `geodatamine` to fetch the tourist attractions and display them on a map. Please note that the available data and characteristics (like price) depend on what's available in the OpenStreetMap data for that specific location and category.

In [ ]:
geolocator = Nominatim(user_agent="city_mapper")
location = geolocator.geocode(city_name)

if location:
    # Create a map centered at the city
    m = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)

    # Add a marker for the city
    folium.Marker(
        location=[location.latitude, location.longitude],
        popup=city_name,
        icon=folium.Icon(color='blue')
    ).add_to(m)

    # Display the map
    display(m)
else:
    print(f"Could not find coordinates for the city: {city_name}")

In [ ]:
from geopy.geocoders import Nominatim
import folium
import pandas as pd
import requests # Import the requests library to query the Overpass API

# Get city name from user input
city_name = input("Enter the name of the city: ")

# Geocode the city name to get coordinates and bounding box
geolocator = Nominatim(user_agent="city_mapper")
location = geolocator.geocode(city_name, exactly_one=True)

if location:
    # Get the bounding box of the city
    # The bounding box is usually available in the 'raw' data from geopy
    if 'boundingbox' in location.raw:
        bbox = location.raw['boundingbox']
        # The bounding box from Nominatim is typically [south, north, west, east]
        south, north, west, east = [float(x) for x in bbox]

        # Create a map centered at the city
        m = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)

        # Add a marker for the city
        folium.Marker(
            location=[location.latitude, location.longitude],
            popup=city_name,
            icon=folium.Icon(color='blue')
        ).add_to(m)

        # --- Code to fetch data from Overpass API ---
        # Overpass API endpoint
        overpass_url = "http://overpass-api.de/api/interpreter"

        # Overpass query to get tourist attractions (restaurants, museums, etc.) within the bounding box
        # This is a basic query, you can refine it based on specific tags
        overpass_query = f"""
        [out:json];
        (
          node["amenity"="restaurant"]({south},{west},{north},{east});
          way["amenity"="restaurant"]({south},{west},{north},{east});
          relation["amenity"="restaurant"]({south},{west},{north},{east});
          node["tourism"="museum"]({south},{west},{north},{east});
          way["tourism"="museum"]({south},{west},{north},{east});
          relation["tourism"="museum"]({south},{west},{north},{east});
          node["tourism"="attraction"]({south},{west},{north},{east});
          way["tourism"="attraction"]({south},{west},{north},{east});
          relation["tourism"="attraction"]({south},{west},{north},{east});
        );
        out center;
        """

        try:
            response = requests.get(overpass_url, params={'data': overpass_query})
            response.raise_for_status() # Raise an exception for bad status codes
            data = response.json()

            # Process the data and add markers to the map
            if data and 'elements' in data:
                tourist_attractions_list = []
                for element in data['elements']:
                    # Use 'lat' and 'lon' for nodes, and 'center' for ways/relations
                    if element['type'] == 'node':
                        lat = element['lat']
                        lon = element['lon']
                    elif 'center' in element:
                        lat = element['center']['lat']
                        lon = element['center']['lon']
                    else:
                        continue # Skip elements without coordinates

                    name = element.get('tags', {}).get('name', 'Unnamed')
                    tags = element.get('tags', {})

                    # Construct popup HTML with relevant characteristics
                    popup_html = f"<b>{name}</b><br>"

                    # Add price/fee information if available
                    price_info = tags.get('price', tags.get('cost', tags.get('fee', 'N/A')))
                    popup_html += f"Price/Fee: {price_info}<br>"

                    # Add other relevant characteristics, excluding those already handled and diet tags for restaurants
                    for key, value in tags.items():
                        # Exclude common tags and diet tags if it's a restaurant
                        if key not in ['name', 'website', 'phone', 'price', 'cost', 'fee', 'amenity', 'tourism']:
                             if tags.get('amenity') == 'restaurant' and key.startswith('diet:'):
                                continue # Skip diet tags for restaurants
                             popup_html += f"{key}: {value}<br>"


                    # Determine marker color based on type
                    marker_color = 'green' # Default color
                    if tags.get('amenity') == 'restaurant':
                        marker_color = 'red'
                    elif tags.get('tourism') == 'museum':
                        marker_color = 'blue'


                    tourist_attractions_list.append({
                        'name': name,
                        'latitude': lat,
                        'longitude': lon,
                        'tags': tags # Store all tags for potential future use
                    })

                    folium.Marker(
                        location=[lat, lon],
                        popup=popup_html,
                        icon=folium.Icon(color=marker_color)
                    ).add_to(m)

                # Optionally, create a DataFrame from the collected data
                tourist_attractions_df = pd.DataFrame(tourist_attractions_list)
                if not tourist_attractions_df.empty:
                    print("Tourist Attractions Found:")
                    display(tourist_attractions_df)
                else:
                     print("No tourist attractions found matching the query.")


            else:
                print("No data elements found in Overpass API response.")

        except requests.exceptions.RequestException as e:
            print(f"Error querying Overpass API: {e}")
        except Exception as e:
            print(f"An error occurred while processing Overpass data: {e}")

        # Display the map
        display(m)
    else:
        print(f"Could not get bounding box for city: {city_name}")

else:
    print(f"Could not find coordinates for the city: {city_name}")

Enter the name of the city: nice
Tourist Attractions Found:


,name,latitude,longitude,tags
0,Le Kobé,43.699512,7.278087,"{'amenity': 'restaurant', 'cuisine': 'asian', ..."
1,La Maison Libanaise,43.708402,7.261123,"{'amenity': 'restaurant', 'cuisine': 'lebanese..."
2,Au grill d'or,43.708053,7.259973,"{'amenity': 'restaurant', 'name': 'Au grill d'..."
3,Le Castello,43.707009,7.257123,"{'amenity': 'restaurant', 'check_date': '2023-..."
4,La Voglia,43.695609,7.273396,"{'amenity': 'restaurant', 'diet:vegetarian': '..."
...,...,...,...,...
878,Des Pizzas Mais Pas Que...,43.710574,7.291765,"{'addr:housenumber': '22', 'addr:street': 'Rue..."
879,Régence Plage,43.684331,7.235842,"{'addr:housenumber': '223', 'addr:street': 'Pr..."
880,Musée Prieuré du Vieux Logis,43.720628,7.254972,"{'addr:housenumber': '59', 'addr:street': 'Ave..."
881,Musée d'art moderne et d'art contemporain,43.701446,7.278500,"{'addr:city': 'Nice', 'addr:postcode': '06300'..."


###Partie 2

In [ ]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import folium
import requests
import time
from datetime import datetime, timedelta

In [ ]:
def trouver_destinations_par_temps(ville_depart, temps_trajet_max_str):
    """
    Trouve les villes accessibles depuis une ville de départ dans un temps de trajet donné.

    Args:
        ville_depart (str): La ville de départ.
        temps_trajet_max_str (str): Le temps de trajet max au format "HH:MM:SS" (ex: "02:30:00").

    Returns:
        list: Une liste de tuples (Origine, Destination, Durée).
    """
    pattern = f"%{ville_depart}%"
    sql = """
    SELECT
        Origine,
        Destination,
        strftime('%H:%M:%S', (julianday(Heure_arrivee) - julianday(Heure_depart)) * 86400, 'unixepoch') AS duree
    FROM
        tgvmax_trajets
    WHERE
        LOWER(Origine) LIKE LOWER(?)
        AND duree <= ?
    ORDER BY
        duree
    """
    cur.execute(sql, (pattern, temps_trajet_max_str))
    resultats = cur.fetchall()
    return resultats

# --- Exemple d'utilisation ---
ville_dep = "PARIS" # Mettez votre ville ici
temps_max = "02:00:00" # Je veux aller quelque part en moins de 2 heures

destinations_possibles = trouver_destinations_par_temps(ville_dep, temps_max)

if destinations_possibles:
    print(f"Destinations possibles depuis {ville_dep} en moins de {temps_max} :")
    df_destinations = pd.DataFrame(destinations_possibles, columns=['Origine', 'Destination', 'Durée'])
    display(df_destinations)
else:
    print("Aucune destination trouvée dans ce laps de temps.")

Destinations possibles depuis PARIS en moins de 02:00:00 :


,Origine,Destination,Durée
0,PARIS (intramuros),CHAMPAGNE ARDENNE TGV,00:39:00
1,PARIS (intramuros),CHAMPAGNE ARDENNE TGV,00:39:00
2,PARIS (intramuros),CHAMPAGNE ARDENNE TGV,00:39:00
3,PARIS (intramuros),CHAMPAGNE ARDENNE TGV,00:39:00
4,PARIS (intramuros),CHAMPAGNE ARDENNE TGV,00:39:00
...,...,...,...
3678,PARIS (intramuros),STRASBOURG,02:00:00
3679,PARIS (intramuros),NEVERS,02:00:00
3680,PARIS (intramuros),FUTUROSCOPE,02:00:00
3681,PARIS (intramuros),FUTUROSCOPE,02:00:00


In [ ]:


def proposer_aller_retour(ville_depart, ville_arrivee, heure_depart_souhaitee_str, temps_sur_place_str):
    """
    Propose un trajet aller et un trajet retour optimaux.

    Args:
        ville_depart (str): La ville de départ.
        ville_arrivee (str): La ville d'arrivée.
        heure_depart_souhaitee_str (str): Heure de départ souhaitée ("HH:MM").
        temps_sur_place_str (str): Temps à passer sur place ("HH:MM").

    Returns:
        dict: Un dictionnaire contenant les informations du trajet aller et retour.
    """
    # --- TROUVER LE TRAJET ALLER ---
    pattern_dep = f"%{ville_depart}%"
    pattern_arr = f"%{ville_arrivee}%"
    sql_aller = """
    SELECT
        Origine, Destination, TIME(Heure_depart) AS heure_depart, TIME(Heure_arrivee) AS heure_arrivee
    FROM
        tgvmax_trajets
    WHERE
        LOWER(Origine) LIKE LOWER(?) AND LOWER(Destination) LIKE LOWER(?) AND TIME(Heure_depart) >= ?
    ORDER BY
        heure_depart ASC
    LIMIT 1
    """
    cur.execute(sql_aller, (pattern_dep, pattern_arr, heure_depart_souhaitee_str))
    trajet_aller = cur.fetchone()

    if not trajet_aller:
        return {"erreur": "Aucun trajet aller trouvé après l'heure souhaitée."}

    # --- CALCULER L'HEURE DE DÉPART POUR LE RETOUR ---
    heure_arrivee_aller = datetime.strptime(trajet_aller[3], '%H:%M:%S')
    temps_sur_place = datetime.strptime(temps_sur_place_str, '%H:%M')
    delta_temps_sur_place = timedelta(hours=temps_sur_place.hour, minutes=temps_sur_place.minute)
    heure_depart_retour_min = (heure_arrivee_aller + delta_temps_sur_place).strftime('%H:%M:%S')

    # --- TROUVER LE TRAJET RETOUR ---
    sql_retour = """
    SELECT
        Origine, Destination, TIME(Heure_depart) AS heure_depart, TIME(Heure_arrivee) AS heure_arrivee
    FROM
        tgvmax_trajets
    WHERE
        LOWER(Origine) LIKE LOWER(?) AND LOWER(Destination) LIKE LOWER(?) AND TIME(Heure_depart) >= ?
    ORDER BY
        heure_depart ASC
    LIMIT 1
    """
    cur.execute(sql_retour, (pattern_arr, pattern_dep, heure_depart_retour_min))
    trajet_retour = cur.fetchone()

    if not trajet_retour:
        return {"erreur": "Aucun trajet retour trouvé après le temps sur place."}

    return {
        "aller": trajet_aller,
        "retour": trajet_retour
    }

# --- Exemple d'utilisation ---
itineraire = proposer_aller_retour("PARIS", "LYON", "09:00", "05:00") # Départ 9h, 5h sur place

if "erreur" in itineraire:
    print(itineraire["erreur"])
else:
    print("Itinéraire Suggéré :")
    print(f"  Aller  : Départ de {itineraire['aller'][0]} à {itineraire['aller'][2]}, arrivée à {itineraire['aller'][1]} à {itineraire['aller'][3]}")
    print(f"  Retour : Départ de {itineraire['retour'][0]} à {itineraire['retour'][2]}, arrivée à {itineraire['retour'][1]} à {itineraire['retour'][3]}")

Itinéraire Suggéré :
  Aller  : Départ de PARIS (intramuros) à 09:41:00, arrivée à LYON ST EXUPERY TGV. à 11:33:00
  Retour : Départ de LYON (intramuros) à 16:50:00, arrivée à PARIS (intramuros) à 19:00:00


In [ ]:

def estimer_temps_visite(tags):
    """ Estime le temps de visite en minutes basé sur les tags OSM. """
    if tags.get('tourism') == 'museum' or tags.get('historic') == 'castle':
        return 120
    if tags.get('historic') in ['cathedral', 'church']:
        return 45
    if tags.get('tourism') == 'attraction' or tags.get('historic') == 'monument':
        return 30
    if tags.get('leisure') == 'park':
        return 60
    return 20 # 20 minutes par défaut pour les autres points d'intérêt

def get_lieux_touristiques(nom_ville):
    """ Récupère les lieux touristiques d'une ville via Overpass API. """
    # Initialize geolocator within the function scope
    geolocator = Nominatim(user_agent="city_mapper", timeout=10)

    location = geolocator.geocode(nom_ville, exactly_one=True)
    if not location or 'boundingbox' not in location.raw:
        print(f"Impossible de trouver les coordonnées/limites pour {nom_ville}")
        return []

    bbox = location.raw['boundingbox']
    south, north, west, east = [float(x) for x in bbox]

    overpass_url = "http://overpass-api.de/api/interpreter"
    # Requête plus complète pour plus de lieux
    overpass_query = f"""
    [out:json][timeout:25];
    (
      node["tourism"~"museum|attraction|gallery|viewpoint"]({south},{west},{north},{east});
      way["tourism"~"museum|attraction|gallery|viewpoint"]({south},{west},{north},{east});
      node["historic"~"castle|monument|ruins|cathedral|church"]({south},{west},{north},{east});
      way["historic"~"castle|monument|ruins|cathedral|church"]({south},{west},{north},{east});
    );
    out center;
    """
    try:
        response = requests.get(overpass_url, params={'data': overpass_query})
        response.raise_for_status()
        data = response.json()

        lieux = []
        for element in data.get('elements', []):
            tags = element.get('tags', {})
            name = tags.get('name', 'Sans nom')

            lat, lon = (0, 0)
            if element['type'] == 'node':
                lat, lon = element['lat'], element['lon']
            elif 'center' in element:
                lat = element['center']['lat']
                lon = element['center']['lon']
            else:
                continue

            # Ajout de l'estimation du temps et d'un score (simple pour le moment)
            temps = estimer_temps_visite(tags)
            score = tags.get('wikipedia') is not None # Le lieu a une page Wikipedia ? C'est un bon indicateur d'importance.

            lieux.append({
                'nom': name,
                'latitude': lat,
                'longitude': lon,
                'tags': tags,
                'temps_visite_min': temps,
                'score_pertinence': 1 if score else 0 # 1 si page wiki, 0 sinon
            })
        return lieux
    except requests.exceptions.RequestException as e:
        print(f"Erreur API Overpass : {e}")
        return []

# --- Exemple d'utilisation ---
lieux_a_lyon = get_lieux_touristiques("toulouse matabiau")
if lieux_a_lyon:
    df_lieux = pd.DataFrame(lieux_a_lyon)
    # Trier par pertinence (score) puis par nom
    df_lieux_tries = df_lieux.sort_values(by=['score_pertinence', 'nom'], ascending=[False, True])
    print("Lieux touristiques trouvés à Lyon :")
    display(df_lieux_tries.head(10)) # Afficher les 10 plus pertinents
else:
    print("Aucun lieu touristique trouvé.")

Lieux touristiques trouvés à Lyon :


,nom,latitude,longitude,tags,temps_visite_min,score_pertinence
2,Immeuble des Cariatides,43.608034,1.451434,"{'building': 'yes', 'heritage': '3', 'heritage...",30,0
1,L'Imagerie,43.615704,1.456401,"{'addr:city': 'Toulouse', 'addr:postcode': '31...",20,0
0,Sans nom,43.610049,1.455645,"{'level': '3', 'tourism': 'viewpoint'}",20,0


In [ ]:
def calculer_temps_trajet_a_pied(coords1, coords2, vitesse_kmh=4.5):
    """
    Calcule le temps de trajet à pied en minutes entre deux points GPS.

    Args:
        coords1 (tuple): (latitude, longitude) du point de départ.
        coords2 (tuple): (latitude, longitude) du point d'arrivée.
        vitesse_kmh (float): Vitesse de marche moyenne en km/h.

    Returns:
        float: Le temps de trajet estimé en minutes.
    """
    if not coords1 or not coords2:
        return 0

    # Calcule la distance en kilomètres
    distance_km = geodesic(coords1, coords2).kilometers

    # Calcule le temps en heures, puis le convertit en minutes
    temps_heures = distance_km / vitesse_kmh
    temps_minutes = temps_heures * 60

    return round(temps_minutes)

In [ ]:
def creer_itineraire_visite_avec_trajet(lieux_tries, temps_disponible_min):
    """
    Crée un itinéraire de visite en sélectionnant les lieux les plus pertinents,
    en incluant le temps de trajet à pied entre chaque lieu.
    """
    itineraire = []
    temps_total = 0

    if not lieux_tries:
        return [], 0

    # 1. Ajouter le premier lieu (le plus important), sans temps de trajet initial
    premier_lieu = lieux_tries[0]
    if premier_lieu['temps_visite_min'] <= temps_disponible_min:
        itineraire.append(premier_lieu)
        temps_total += premier_lieu['temps_visite_min']
    else:
        # Si on n'a même pas le temps de visiter le premier lieu, on s'arrête.
        return [], 0

    # 2. Parcourir le reste des lieux pour voir si on peut les ajouter
    for lieu_candidat in lieux_tries[1:]:
        # Le point de départ est le dernier lieu ajouté à notre itinéraire
        dernier_lieu_visite = itineraire[-1]

        coords_dernier_lieu = (dernier_lieu_visite['latitude'], dernier_lieu_visite['longitude'])
        coords_candidat = (lieu_candidat['latitude'], lieu_candidat['longitude'])

        # 3. Calculer le temps de trajet
        temps_trajet = calculer_temps_trajet_a_pied(coords_dernier_lieu, coords_candidat)

        temps_visite_candidat = lieu_candidat['temps_visite_min']

        # 4. Vérifier si on a le temps d'y aller ET de le visiter
        if temps_total + temps_trajet + temps_visite_candidat <= temps_disponible_min:
            # On stocke le temps de trajet pour l'afficher plus tard
            lieu_candidat['trajet_depuis_precedent'] = temps_trajet
            itineraire.append(lieu_candidat)
            temps_total += temps_trajet + temps_visite_candidat

    return itineraire, temps_total

# --- Exemple d'utilisation (à placer après avoir récupéré les lieux de Lyon) ---

# On suppose que df_lieux_tries existe et a été créé comme dans la réponse précédente
temps_sur_place_a_lyon_min = 5 * 60 # 5 heures

lieux_tries_list = df_lieux_tries.to_dict('records')
itineraire_lyon, temps_utilise = creer_itineraire_visite_avec_trajet(lieux_tries_list, temps_sur_place_a_lyon_min)

if itineraire_lyon:
    print(f"Itinéraire de visite OPTIMISÉ pour Lyon ({temps_utilise / 60:.1f}h sur {temps_sur_place_a_lyon_min / 60:.1f}h disponibles):")

    # Affiche le premier lieu
    lieu = itineraire_lyon[0]
    print(f"  1. {lieu['nom']} (Visite : {lieu['temps_visite_min']} min)")

    # Affiche les lieux suivants avec le temps de trajet
    for i, lieu in enumerate(itineraire_lyon[1:], start=2):
        print(f"     -> Trajet à pied : {lieu['trajet_depuis_precedent']} min")
        print(f"  {i}. {lieu['nom']} (Visite : {lieu['temps_visite_min']} min)")

    print(f"\nTemps total estimé (visites + trajets) : {temps_utilise} minutes.")
else:
    print("Pas assez de temps pour visiter des lieux.")

Itinéraire de visite OPTIMISÉ pour Lyon (4.6h sur 5.0h disponibles):
  1. A400M (Visite : 30 min)
     -> Trajet à pied : 2 min
  2. ATR 42 (Visite : 30 min)
     -> Trajet à pied : 1 min
  3. ATR72 (Visite : 30 min)
     -> Trajet à pied : 1 min
  4. Aeroscopia (Visite : 120 min)
     -> Trajet à pied : 2 min
  5. Airbus A320 (Visite : 30 min)
     -> Trajet à pied : 1 min
  6. Airbus A340-600 (Visite : 30 min)

Temps total estimé (visites + trajets) : 277 minutes.


In [ ]:
#voir fonction ci-dessus

def creer_itineraire_visite(lieux_tries, temps_disponible_min):
    """
    Crée un itinéraire de visite en sélectionnant les lieux les plus pertinents
    dans la limite du temps disponible.
    """
    itineraire = []
    temps_total = 0
    for lieu in lieux_tries:
        if temps_total + lieu['temps_visite_min'] <= temps_disponible_min:
            itineraire.append(lieu)
            temps_total += lieu['temps_visite_min']
    return itineraire, temps_total

# --- Exemple d'utilisation ---
temps_sur_place_a_lyon_min = 5 * 60 # 5 heures

# On réutilise le dataframe trié de l'étape précédente
lieux_tries_list = df_lieux_tries.to_dict('records')
itineraire_lyon, temps_utilise = creer_itineraire_visite(lieux_tries_list, temps_sur_place_a_lyon_min)

if itineraire_lyon:
    print(f"Itinéraire de visite proposé pour Lyon ({temps_utilise / 60:.1f}h sur {temps_sur_place_a_lyon_min / 60:.1f}h disponibles):")
    for i, lieu in enumerate(itineraire_lyon):
        print(f"  {i+1}. {lieu['nom']} (Visite estimée : {lieu['temps_visite_min']} min)")
else:
    print("Pas assez de temps pour visiter des lieux.")

Itinéraire de visite proposé pour Lyon (5.0h sur 5.0h disponibles):
  1. A400M (Visite estimée : 30 min)
  2. ATR 42 (Visite estimée : 30 min)
  3. ATR72 (Visite estimée : 30 min)
  4. Aeroscopia (Visite estimée : 120 min)
  5. Airbus A320 (Visite estimée : 30 min)
  6. Airbus A340-600 (Visite estimée : 30 min)
  7. Airbus A380 (Visite estimée : 30 min)


In [ ]:
def generer_carte_recommandation(ville_depart, destinations, itineraire_choisi, ville_choisie, geolocator):
    """
    Génère une carte Folium avec la ville de départ, les destinations possibles et
    l'itinéraire de visite pour la destination recommandée.
    """
    # Centrer la carte sur la ville de départ
    loc_depart = geolocator.geocode(ville_depart)
    if not loc_depart:
        return "Impossible de géolocaliser la ville de départ."

    m = folium.Map(location=[loc_depart.latitude, loc_depart.longitude], zoom_start=7)

    # Marqueur pour le départ
    folium.Marker(
        location=[loc_depart.latitude, loc_depart.longitude],
        popup=f"<b>Départ : {ville_depart}</b>",
        icon=folium.Icon(color='red', icon='train', prefix='fa')
    ).add_to(m)

    # Marqueurs pour les autres destinations possibles
    for dest in destinations:
        ville_dest_nom = dest[1]
        if ville_dest_nom != ville_choisie:
            try:
                loc_dest = geolocator.geocode(clean_city_name(ville_dest_nom))
                if loc_dest:
                    folium.Marker(
                        location=[loc_dest.latitude, loc_dest.longitude],
                        popup=f"<i>{ville_dest_nom}</i><br>Durée : {dest[2]}",
                        icon=folium.Icon(color='blue', icon='info-sign')
                    ).add_to(m)
            except: continue

    # Marqueurs pour l'itinéraire recommandé dans la ville choisie
    for i, lieu in enumerate(itineraire_choisi):
        popup_html = f"<b>{i+1}. {lieu['nom']}</b><br>Visite: {lieu['temps_visite_min']} min"
        if 'trajet_depuis_precedent' in lieu:
            popup_html = f"Trajet: {lieu['trajet_depuis_precedent']} min<br>" + popup_html
        folium.Marker(
            location=[lieu['latitude'], lieu['longitude']],
            popup=popup_html,
            icon=folium.Icon(color='green', icon='camera', prefix='fa')
        ).add_to(m)

    return m

# --- EXEMPLE DE SYNTHÈSE COMPLÈTE ---
# 1. Définir les conditions de l'utilisateur
VILLE_DEPART = "marmande"
TEMPS_TRAJET_MAX = "02:00:00" # Moins de 2h00 de trajet
TEMPS_SUR_PLACE_MIN = 6 * 60 # 6 heures de visite

print("=============================================")
print(f"🚀 Lancement de la recherche d'itinéraire")
print(f"   Ville de départ : {VILLE_DEPART}")
print(f"   Temps de trajet max : {TEMPS_TRAJET_MAX}")
print(f"   Temps sur place : {TEMPS_SUR_PLACE_MIN / 60} heures")
print("=============================================\n")


# 2. Trouver toutes les destinations qui correspondent
destinations_candidates = trouver_destinations_par_temps(VILLE_DEPART, TEMPS_TRAJET_MAX)
destinations_uniques = sorted(list(set([d[1] for d in destinations_candidates]))) # Garder seulement les noms de villes uniques
print(f"🔎 {len(destinations_uniques)} destinations uniques trouvées en moins de {TEMPS_TRAJET_MAX} de trajet.\n")

# 3. Évaluer chaque destination
meilleure_destination = None
meilleur_itineraire = []
max_score = -1

for ville in destinations_uniques:
    print(f"🏙️  Analyse de : {ville}...")
    lieux = get_lieux_touristiques(ville)

    if not lieux:
        print("   -> Aucun lieu touristique trouvé.\n")
        continue

    # Trier les lieux par pertinence (score wikipedia)
    lieux_tries = sorted(lieux, key=lambda x: x['score_pertinence'], reverse=True)

    # Créer un itinéraire optimisé
    itineraire_ville, temps_total_visite = creer_itineraire_visite_avec_trajet(lieux_tries, TEMPS_SUR_PLACE_MIN)

    # "Noter" cette destination par le nombre de lieux visitables (critère simple mais efficace)
    score_actuel = len(itineraire_ville)
    print(f"   -> Score : {score_actuel} activités possibles en {temps_total_visite} min.")

    if score_actuel > max_score:
        max_score = score_actuel
        meilleure_destination = ville
        meilleur_itineraire = itineraire_ville
    print("-" * 20)


# 4. Afficher la recommandation
if meilleure_destination:
    print("\n\n=============================================")
    print("🏆 RECOMMANDATION N°1 🏆")
    print(f"La meilleure destination est : {meilleure_destination} avec {max_score} activités possibles !")
    print("=============================================\n")
    print("📋 Itinéraire de visite suggéré sur place :")

    lieu = meilleur_itineraire[0]
    print(f"  1. {lieu['nom']} (Visite : {lieu['temps_visite_min']} min)")
    for i, lieu in enumerate(meilleur_itineraire[1:], start=2):
        print(f"     -> Trajet à pied : ~{lieu['trajet_depuis_precedent']} min")
        print(f"  {i}. {lieu['nom']} (Visite : {lieu['temps_visite_min']} min)")

    # 5. Générer la carte finale
    print("\n🗺️  Génération de la carte...")
    carte_finale = generer_carte_recommandation(VILLE_DEPART, destinations_candidates, meilleur_itineraire, meilleure_destination, geolocator)
    display(carte_finale)
else:
    print("\n❌ Désolé, aucune destination n'a été trouvée correspondant à tous vos critères.")

🚀 Lancement de la recherche d'itinéraire
   Ville de départ : marmande
   Temps de trajet max : 02:00:00
   Temps sur place : 6.0 heures

🔎 3 destinations uniques trouvées en moins de 02:00:00 de trajet.

🏙️  Analyse de : BORDEAUX ST JEAN...
   -> Score : 1 activités possibles en 120 min.
--------------------
🏙️  Analyse de : MONTAUBAN VILLE BOURBON...
   -> Aucun lieu touristique trouvé.

🏙️  Analyse de : TOULOUSE MATABIAU...
   -> Score : 3 activités possibles en 91 min.
--------------------


🏆 RECOMMANDATION N°1 🏆
La meilleure destination est : TOULOUSE MATABIAU avec 3 activités possibles !

📋 Itinéraire de visite suggéré sur place :
  1. Sans nom (Visite : 20 min)
     -> Trajet à pied : ~8 min
  2. L'Imagerie (Visite : 20 min)
     -> Trajet à pied : ~13 min
  3. Immeuble des Cariatides (Visite : 30 min)

🗺️  Génération de la carte...


In [ ]:
# ==============================================================================
# BLOC 1 : INSTALLATION, IMPORTS ET CONFIGURATION
# ==============================================================================

# --- Installation des bibliothèques ---
!pip install geopy folium requests pandas -q

# --- Imports ---
import sqlite3
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import folium
import requests
import re
from IPython.display import display

# --- Connexion à la base de données ---
# !!! ATTENTION : Modifiez ce chemin si nécessaire !!!
db_path = "/content/drive/MyDrive/Colab Notebook/SNCF/tgvmax.db"
try:
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    print("✅ Connexion à la base de données réussie.")
except Exception as e:
    print(f"❌ Erreur de connexion à la base de données : {e}")
    print("Veuillez vérifier que le chemin d'accès est correct et que votre Drive est monté.")

# --- Initialisation du géocodeur ---
geolocator = Nominatim(user_agent="mon_appli_itineraire_v2")

# ==============================================================================
# BLOC 2 : FONCTIONS UTILITAIRES (HELPERS)
# ==============================================================================

def clean_city_name(city_name):
    """Nettoie le nom d'une ville en supprimant les mentions comme '(intramuros)'."""
    if not isinstance(city_name, str):
        return ""
    # Remove "(intramuros)" and any leading/trailing whitespace
    cleaned_name = re.sub(r'\s*\([^)]*\)$', '', city_name).strip()
    return cleaned_name

def estimer_temps_visite(tags):
    """Estime le temps de visite en minutes basé sur les tags OSM."""
    if tags.get('tourism') == 'museum' or tags.get('historic') == 'castle':
        return 120  # 2 heures
    if tags.get('historic') in ['cathedral', 'church']:
        return 45   # 45 minutes
    if tags.get('tourism') == 'attraction' or tags.get('historic') == 'monument':
        return 30   # 30 minutes
    if tags.get('leisure') == 'park':
        return 60   # 1 heure
    return 20 # 20 minutes par défaut

def calculer_temps_trajet_a_pied(coords1, coords2, vitesse_kmh=4.5):
    """Calcule le temps de trajet à pied en minutes entre deux points GPS."""
    if not coords1 or not coords2:
        return 0
    distance_km = geodesic(coords1, coords2).kilometers
    temps_minutes = (distance_km / vitesse_kmh) * 60
    return round(temps_minutes)

# ==============================================================================
# BLOC 3 : FONCTIONS PRINCIPALES (LOGIQUE DU PROJET)
# ==============================================================================

def trouver_destinations_par_temps(ville_depart, temps_trajet_max_str):
    """Trouve les villes accessibles depuis une ville de départ dans un temps de trajet donné."""
    pattern = f"%{clean_city_name(ville_depart)}%"
    sql = """
    SELECT DISTINCT Origine, Destination,
           strftime('%H:%M:%S', (julianday(Heure_arrivee) - julianday(Heure_depart)) * 86400, 'unixepoch') AS duree
    FROM tgvmax_trajets
    WHERE LOWER(Origine) LIKE LOWER(?) AND duree > '00:00:00' AND duree <= ?
    ORDER BY duree
    """
    cur.execute(sql, (pattern, temps_trajet_max_str))
    return cur.fetchall()

def get_lieux_touristiques(nom_ville):
    """Récupère les lieux touristiques d'une ville via Overpass API."""
    nom_ville_nettoye = clean_city_name(nom_ville)
    try:
        location = geolocator.geocode(nom_ville_nettoye, exactly_one=True, timeout=10)
        if not location or 'boundingbox' not in location.raw:
            print(f"   -> Info: Impossible de trouver les limites pour {nom_ville_nettoye}")
            return []
    except Exception as e:
        print(f"   -> Erreur de géocodage pour {nom_ville_nettoye}: {e}")
        return []

    bbox = location.raw['boundingbox']
    south, north, west, east = [float(x) for x in bbox]
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json][timeout:25];
    (
      node["tourism"~"museum|attraction|gallery|viewpoint"]({south},{west},{north},{east});
      way["tourism"~"museum|attraction|gallery|viewpoint"]({south},{west},{north},{east});
      node["historic"~"castle|monument|ruins|cathedral|church"]({south},{west},{north},{east});
      way["historic"~"castle|monument|ruins|cathedral|church"]({south},{west},{north},{east});
    );
    out center;
    """
    try:
        response = requests.get(overpass_url, params={'data': overpass_query})
        response.raise_for_status()
        data = response.json()
        lieux = []
        for element in data.get('elements', []):
            tags = element.get('tags', {})
            if 'name' not in tags: continue # On ignore les lieux sans nom

            lat, lon = (element.get('lat'), element.get('lon')) if element['type'] == 'node' else (element.get('center', {}).get('lat'), element.get('center', {}).get('lon'))
            if not lat or not lon: continue

            lieux.append({
                'nom': tags['name'], 'latitude': lat, 'longitude': lon, 'tags': tags,
                'temps_visite_min': estimer_temps_visite(tags),
                'score_pertinence': 1 if 'wikipedia' in tags else 0
            })
        return lieux
    except requests.exceptions.RequestException as e:
        print(f"   -> Erreur API Overpass pour {nom_ville_nettoye}: {e}")
        return []

def creer_itineraire_visite_avec_trajet(lieux_tries, temps_disponible_min):
    """Crée un itinéraire de visite en incluant le temps de trajet à pied."""
    itineraire = []
    temps_total = 0
    if not lieux_tries: return [], 0

    premier_lieu = lieux_tries[0]
    if premier_lieu['temps_visite_min'] <= temps_disponible_min:
        itineraire.append(premier_lieu)
        temps_total += premier_lieu['temps_visite_min']
    else:
        return [], 0

    for lieu_candidat in lieux_tries[1:]:
        dernier_lieu_visite = itineraire[-1]
        coords_dernier_lieu = (dernier_lieu_visite['latitude'], dernier_lieu_visite['longitude'])
        coords_candidat = (lieu_candidat['latitude'], lieu_candidat['longitude'])
        temps_trajet = calculer_temps_trajet_a_pied(coords_dernier_lieu, coords_candidat)

        if temps_total + temps_trajet + lieu_candidat['temps_visite_min'] <= temps_disponible_min:
            lieu_candidat['trajet_depuis_precedent'] = temps_trajet
            itineraire.append(lieu_candidat)
            temps_total += temps_trajet + lieu_candidat['temps_visite_min']

    return itineraire, temps_total

def generer_carte_recommandation(ville_depart, destinations, itineraire_choisi, ville_choisie, geolocator):
    """Génère la carte Folium finale avec toutes les informations."""
    ville_depart_nettoyee = clean_city_name(ville_depart)
    loc_depart = geolocator.geocode(ville_depart_nettoyee)
    if not loc_depart:
        return "Impossible de géolocaliser la ville de départ."

    m = folium.Map(location=[loc_depart.latitude, loc_depart.longitude], zoom_start=7)

    folium.Marker(
        location=[loc_depart.latitude, loc_depart.longitude],
        popup=f"<b>Départ : {ville_depart}</b>",
        icon=folium.Icon(color='red', icon='train', prefix='fa')
    ).add_to(m)

    # Add a red circle around the recommended destination
    loc_choisie = geolocator.geocode(clean_city_name(ville_choisie))
    if loc_choisie:
        folium.Circle(
            location=[loc_choisie.latitude, loc_choisie.longitude],
            radius=5000,  # Radius in meters (adjust as needed for visibility)
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.2,
            popup=f"Destination recommandée: {ville_choisie}"
        ).add_to(m)


    for dest in destinations:
        ville_dest_nom = dest[1]
        if ville_dest_nom != ville_choisie:
            try:
                loc_dest = geolocator.geocode(clean_city_name(ville_dest_nom))
                if loc_dest:
                    folium.Marker(
                        location=[loc_dest.latitude, loc_dest.longitude],
                        popup=f"<i>{ville_dest_nom}</i><br>Durée : {dest[2]}",
                        icon=folium.Icon(color='blue', icon='info-sign')
                    ).add_to(m)
            except: continue


    # Marqueurs pour l'itinéraire recommandé
    for i, lieu in enumerate(itineraire_choisi):
        popup_html = f"<b>{i+1}. {lieu['nom']}</b><br>Visite: {lieu['temps_visite_min']} min"
        if 'trajet_depuis_precedent' in lieu:
            popup_html = f"Trajet: {lieu['trajet_depuis_precedent']} min<br>" + popup_html
        folium.Marker(
            location=[lieu['latitude'], lieu['longitude']],
            popup=popup_html,
            icon=folium.Icon(color='green', icon='camera', prefix='fa')
        ).add_to(m)

    return m

# ==============================================================================
# BLOC 4 : EXÉCUTION DU TEST AVEC "PARIS"
# ==============================================================================

# --- 1. Définir les conditions de l'utilisateur (avec inputs) ---
VILLE_DEPART = input("Entrez votre ville de départ : ")
TEMPS_TRAJET_MAX = input("Entrez le temps de trajet maximum souhaité (HH:MM:SS) : ")
temps_sur_place_heures = float(input("Entrez le temps que vous souhaitez passer sur place (en heures) : "))
TEMPS_SUR_PLACE_MIN = int(temps_sur_place_heures * 60) # Convertir en minutes


print("=============================================")
print(f"🚀 Lancement de la recherche d'itinéraire")
print(f"   Ville de départ : {VILLE_DEPART}")
print(f"   Temps de trajet max : {TEMPS_TRAJET_MAX}")
print(f"   Temps sur place : {TEMPS_SUR_PLACE_MIN / 60} heures")
print("=============================================\n")


# --- 2. Trouver toutes les destinations candidates ---
destinations_candidates = trouver_destinations_par_temps(VILLE_DEPART, TEMPS_TRAJET_MAX)
destinations_uniques = sorted(list(set([d[1] for d in destinations_candidates])))
print(f"🔎 {len(destinations_uniques)} destinations uniques trouvées en moins de {TEMPS_TRAJET_MAX} de trajet.\n")

# --- 3. Évaluer chaque destination ---
meilleure_destination = None
meilleur_itineraire = []
max_score = -1

for ville in destinations_uniques:
    print(f"🏙️  Analyse de : {ville}...")
    lieux = get_lieux_touristiques(ville)

    if not lieux:
        print("   -> Aucun lieu touristique trouvé.\n")
        continue

    # Trier les lieux par pertinence (score wikipedia)
    lieux_tries = sorted(lieux, key=lambda x: x['score_pertinence'], reverse=True)

    # Créer un itinéraire optimisé
    itineraire_ville, temps_total_visite = creer_itineraire_visite_avec_trajet(lieux_tries, TEMPS_SUR_PLACE_MIN)

    # "Noter" cette destination par le nombre de lieux visitables (critère simple mais efficace)
    score_actuel = len(itineraire_ville)
    print(f"   -> Score : {score_actuel} activités possibles en {temps_total_visite} min.")

    if score_actuel > max_score:
        max_score = score_actuel
        meilleure_destination = ville
        meilleur_itineraire = itineraire_ville
    print("-" * 20)

# --- 4. Afficher la recommandation finale ---
if meilleure_destination:
    print("\n\n=============================================")
    print("🏆 RECOMMANDATION N°1 🏆")
    print(f"La meilleure destination est : {meilleure_destination} avec {max_score} activités possibles !")
    print("=============================================\n")
    print("📋 Itinéraire de visite suggéré sur place :")

    if meilleur_itineraire: # Check if the itinerary is not empty
        lieu = meilleur_itineraire[0]
        print(f"  1. {lieu['nom']} (Visite : {lieu['temps_visite_min']} min)")
        for i, lieu in enumerate(meilleur_itineraire[1:], start=2):
            print(f"     -> Trajet à pied : ~{lieu['trajet_depuis_precedent']} min")
            print(f"  {i}. {lieu['nom']} (Visite : {lieu['temps_visite_min']} min)")
    else:
        print("   Aucun itinéraire de visite détaillé trouvé pour cette destination.")


    # --- 5. Générer et afficher la carte finale ---
    print("\n🗺️  Génération de la carte...")
    carte_finale = generer_carte_recommandation(VILLE_DEPART, destinations_candidates, meilleur_itineraire, meilleure_destination, geolocator)
    display(carte_finale)
else:
    print("\n❌ Désolé, aucune destination n'a été trouvée correspondant à tous vos critères.")

✅ Connexion à la base de données réussie.
Entrez votre ville de départ : marmande
Entrez le temps de trajet maximum souhaité (HH:MM:SS) : 02:00:00
Entrez le temps que vous souhaitez passer sur place (en heures) : 5
🚀 Lancement de la recherche d'itinéraire
   Ville de départ : marmande
   Temps de trajet max : 02:00:00
   Temps sur place : 5.0 heures

🔎 3 destinations uniques trouvées en moins de 02:00:00 de trajet.

🏙️  Analyse de : BORDEAUX ST JEAN...
   -> Score : 1 activités possibles en 120 min.
--------------------
🏙️  Analyse de : MONTAUBAN VILLE BOURBON...
   -> Aucun lieu touristique trouvé.

🏙️  Analyse de : TOULOUSE MATABIAU...
   -> Score : 2 activités possibles en 63 min.
--------------------


🏆 RECOMMANDATION N°1 🏆
La meilleure destination est : TOULOUSE MATABIAU avec 2 activités possibles !

📋 Itinéraire de visite suggéré sur place :
  1. L'Imagerie (Visite : 20 min)
     -> Trajet à pied : ~13 min
  2. Immeuble des Cariatides (Visite : 30 min)

🗺️  Génération de la cart